There were lessons learnt from Amazon, Sourashtramatrimoni, Linkedin websites. There were unique challenges in each cases and the scraping was not exactly very successful. 

There are couple of ideas still not fully explored, 

    Using XSLT or XML libraries to scrape the sources

    Using Xpath to pinpoint the text required

The challenge is only in locating a reliable reference point to the texts in the website sources. It is about time I used the X_path and X_query effectively for my next project. 

The above ideas has been made real in this notebook. The Xpath usage has been greatly streamlined and understood. Following that, much of the ReactJS based website has been scraped with help of Xpath methods

Then extend that to earlier implementations...

In [1]:
#Initiating the Selenium Drivers
from selenium import webdriver
import time as time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dataset
from bs4 import BeautifulSoup 
from lxml import html
import requests
from urllib.parse import unquote
from urllib.parse import urljoin

In [3]:
entry = 'https://www.messari.io'

In [4]:
driver = webdriver.Chrome()
driver.get(entry)
driver.maximize_window()

In [5]:
html_source = driver.page_source
soup_net = BeautifulSoup(html_source,'html.parser')
driver.quit()

In [8]:
soup_lxml = BeautifulSoup(html_source,'lxml')

In [6]:
tree = html.fromstring(html_source) #use the source directly

In [56]:
#This is the long process
#tree.xpath('//section[2]/div/div[2]/div[2]/table/tbody/tr/td/div/span/a/text()')
#crypto_links = tree.xpath('//section[2]/div/div[2]/div[2]/table/tbody/tr/td/div/span/a/@href')

#Trying the shorter Xpath robust process
tbod = tree.xpath('//tbody[contains(@class,"MuiTableBody-root")]')[0]

ecosystem = []
for crypto in tbod.xpath('//a/@href')[11:42]:
    ecosystem.append(urljoin('https://messari.io/',crypto))

In [57]:
cryptos = tbod.xpath('//a/text()')[5:35]
len(cryptos)

ecosystem.pop(15) # It ejects one of the elements from the list
len(ecosystem)

30

In [170]:
#Creating the dataframes with the links and the cryptos/ ecosystem to explore
messari_crawler = pd.DataFrame({'ecosystems':cryptos,'links':ecosystem})
messari_crawler.head()

,ecosystems,links
0,Bitcoin,https://messari.io/asset/bitcoin
1,Ethereum,https://messari.io/asset/ethereum
2,Tether,https://messari.io/asset/tether
3,USD Coin,https://messari.io/asset/usd-coin
4,XRP,https://messari.io/asset/xrp


In [41]:
#Xpath understanding sputters then dies down
tree.xpath('//tbody[@class="MuiTableBody-root jss195 jss768"]/tr[1]/td/div/span/text()')

['$55.06', '$3.38B', '+8.28%']

In [63]:
#let me call one of the crypto profile pages using the selenium, new driver
messari_crawler.links[0]

'https://messari.io/asset/bitcoin'

In [87]:
#none of the below work to get the data required, since the website is ReactJS basis
tree.xpath('//table/@class')
tree.xpath('//section/@class')[:5]
tree.xpath('//span[.="Top Assets"]/text()')
tree.xpath('//div/@class=jss38') 
tree.xpath('//div/@class')[:10]
tree.xpath('//table/tbody[@class="MuiTable-root jss192 jss750"]')
#expression is correct, but no resullt...
tree.xpath('//thead/@*')

[]

In [64]:
crypto_driver = webdriver.Chrome()
crypto_driver.get(messari_crawler.links[0])

1) Find the various major tables or sections on the profile page
    Extract the information from that page section /table

2) Find the tabs on the profile page. 
    get their links

3) Find the Key Areas and list them
    Get the details under key metrics, key info and summary



![](2022-02-21-16-28-05.png)

In [70]:
#failed attempts at getting the top_tabs, don't run

crypto_driver.find_element_by_xpath('//span').text

tabs = crypto_driver.find_elements_by_xpath('//span')
for tab in tabs[12:21]:
    print(tab.text)

Overview
Profile
Markets
Metrics
Historical Data
Charts
Research
Events
News


Tabs to be clicked on every page by the driver, when it crawls are

Overview, 
Profile
Markets
Metrics
Research
News

In [95]:
tabs = crypto_driver.find_elements_by_xpath('//a[@role="tab"]')
profile_page = []
for tab in tabs[1:9]:
    profile_page.append(tab.get_attribute('href'))

In [96]:
profile_page.pop(3)
profile_page.pop(3)
profile_page.pop(4)
profile_page

['https://messari.io/asset/bitcoin/profile',
 'https://messari.io/asset/bitcoin/markets',
 'https://messari.io/asset/bitcoin/metrics',
 'https://messari.io/asset/bitcoin/research',
 'https://messari.io/asset/bitcoin/news']

In [97]:
#Finding the key areas and listing them... which I found were redundant
tabs = crypto_driver.find_elements_by_xpath('//a[not(@role="tab")]')
for tab in tabs[10:22]:
    print(tab.text)

ROI
KEY INFO
SUMMARY
distributed consensus-based
full nodes
miners
game theory
cryptography
PRICE LOOKUP
HISTORICAL DATA




In [126]:
#The ends-with option is not working Chrome-browser so need to use substring, not working
span_data = crypto_driver.find_elements_by_xpath('//span["ion"=substring(@class,string-length(@class),string-length(@class)-3)]')
#//div[substring(@title, string-length(@title) - 2) = 'gle']
#//input['name'=substring(@name,5,8)]
for dat in span_data:
    print(dat.text)

In [162]:
#using the simple array manipulation to get the data
profile_columns = [] #The columns on which the values will be filled 
span_data = crypto_driver.find_elements_by_xpath('//span[contains(@class,"jss")]')
#//div[substring(@title, string-length(@title) - 2) = 'gle']
#//input['name'=substring(@name,5,8)]
for dat in span_data[35:46]:
    profile_columns.append(dat.text)

for dat in span_data[89:95]:
    profile_columns.append(dat.text)

In [163]:
profile_columns.pop(7)

'-43.62%'

In [164]:
profile_columns.pop(6)#down from ATH

'Down from ATH'

In [166]:
profile_columns.pop(8)

'Up From Cycle Low'

In [152]:
span_data = crypto_driver.find_elements_by_xpath('//span[not(contains(@class,"jss"))]')
for dat in span_data[10:25]:
    print(dat.text)

$4.86B
$738B
$803B
$816B
$68,721.93
November 10th, 2021
$33,116.00
January 24th, 2022
Currencies
Native
Payments
Proof-of-Work
Nakamoto Consensus
January 3rd, 2009



In [167]:
len(profile_columns)

14

In [169]:
len(span_data[10:24])

14

In [172]:
for index,cols in enumerate(profile_columns):
    print(cols)
    messari_crawler.loc[0,cols] = span_data[10:25][index].text

Real Volume (24H)
Marketcap
Y+10 Marketcap
Y2050 Marketcap
ATH
ATH Date
Cycle Low
Cycle Low Date
Sector
Token Type
Token Usage
Consensus Algorithm
Precise Consensus Algorithm
Genesis Block Date


In [173]:
messari_crawler.columns

Index(['ecosystems', 'links', 'Real Volume (24H)', 'Marketcap',
       'Y+10 Marketcap', 'Y2050 Marketcap', 'ATH', 'ATH Date', 'Cycle Low',
       'Cycle Low Date', 'Sector', 'Token Type', 'Token Usage',
       'Consensus Algorithm', 'Precise Consensus Algorithm',
       'Genesis Block Date'],
      dtype='object')